In [1]:
# Import necessary libraries
import base64
import json

import os
import fitz
import requests

In [2]:
src = fitz.open("../PyMuPDF_playground/574_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(1)
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page
# r2 = r + (r.width / 2, 0, 0, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page = new_doc.new_page(-1, width=r2.width, height=r2.height)
new_page.show_pdf_page(
    new_page.rect,
    src,
    page.number,
    clip=r2,
)

94

In [3]:
# Get image from PDF
img_list = new_page.get_images(full=True)

for img in img_list:
    rect = new_page.get_image_bbox(img)
    if rect[0] > 0 and rect[1] > 0 and rect[2] > 0 and rect[3] > 0:
        xref = img[0] # get specific xref we want
image = new_doc.extract_image(xref)

# TODO delete manual path definition
# write extracted image to file
image_title = src.name.split('.')[2].split('/')[2]+"_image_"+str(xref)
print(image_title)
image_path = os.path.join(
    '/home/funny/Documents/Projects/230701-Parsing-Tool/augustin-plugin/sample_data/',
    image_title+"."+image['ext']
    )
print(image_title+"."+image['ext'])
with open(
    (image_path),
    "wb",
) as imgout:
    imgout.write(image["image"])


574_augustin_fertig_kontrolle_image_92
574_augustin_fertig_kontrolle_image_92.jpeg


In [4]:
# Upload image 
url = "http://localhost:10004/wp-json/wp/v2/media"

user = "lebe"
password = "gUwM J4pU sngD VHpk Cub7 quS2"

credentials = user + ':' + password

token = base64.b64encode(credentials.encode())

header = {'Authorization': 'Basic ' + token.decode('utf-8')}

media = {'file': open(image_path,'rb'), 'caption': image_title}

response = requests.post(url, headers=header,  files=media)

image_id = json.loads(response.content)['id']

## Extract text

In [17]:
# First, create rect for article text which is on the bottom third of page
r = new_page.rect
r.y0 = (r.y1 * 2) / 3  # two third of page height
article = new_page.get_textbox(r)

In [18]:
# Second, create rect for article title which is in between half and third bottom of page
r2 = new_page.rect
r2.y0 = r2.y1 / 2  # half of page height
r2.y1 = r2.y1 * 2 / 3  - 3 # two third of page height -3 so it is not overlapping
meta_string = new_page.get_textbox(r2)
meta_array = meta_string.splitlines()
meta_array


['Loveth Klinger',
 'Vom Glück, eine Arbeit zu haben',
 'PRoToKoll: SÓnia melo',
 'FoTo: maRio lanG']

In [40]:
# Assign meta data to variables
for i, line in enumerate(meta_array):
	if "protokoll:" in line.lower():
		protocol = meta_array[i].lower().title()
		photograph = meta_array[i+1].lower().title()
		title = meta_array[i-1]
		author = "Autor*in: "+meta_array[i-2].lower().title()

In [41]:
# this gives me the whole part of the article in single letters
# article = []
# return_values
# for index in range(return_values[0], return_values[1]):
# 	article += lines[index]


In [42]:
# str = ''
# for i in article:
#     str += i

# print(str)

In [43]:
# print(article) gives me something like
# ['A', '\n', 'n', 's', 't', 'a', 't', 't', ' ', 'd', 'i', 'e', ' '...]
# As shown, newlines have to be removed correctly i.e. not if a dot is before
article = list(article)
article_edit = article
for index, letter in enumerate(article):
	# deletes newlines that are made up because of articles structure
	if '\n' in letter:
		if '.' in article[index-1] or '!' in article[index-1] or '?' in article[index-1] or ':' in article[index-1]:
			print("supposed to break a new line", letter)
		elif '-' in article[index-1]:
			if ' ' in article[index-1] and ' ' in article[index+1]:
				# this hyphen has been set on purpose and stays
				print("hyphen has been set on purpose")
			else:
				# deletes hyphen on the index and the extra space before it
				del article_edit[index]
				del article_edit[index-1]
		else:
			del article_edit[index]
	# deletes double spaces
	if ' ' in letter and ' ' in article[index+1]:
			del article_edit[index]


supposed to break a new line 

supposed to break a new line 

supposed to break a new line 



In [44]:
# to create a readable text for the article
readable_text = ''
for letter in article_edit:
    readable_text += letter
    
print(readable_text)

Vor zweieinhalb Jahren hatte ich eine schwierige Phase, da lief es nicht so gut für mich, ich war arbeitslos und wegen der Corona- Pandemie konnte ich lange keine Arbeit finden. Bis mir Bekannte von der Möglichkeit erzählten, den Augustin zu verkaufen und so ein bisschen Geld zu verdienen. Dann habe ich eine Einschulung besucht und einen Augustin-Ausweis bekommen, seitdem verkaufe ich die Zeitung, früher am Karlsplatz. Dort war es aber im Winter so kalt! Nun bin ich abwechselnd vor einem Supermarkt bei der U6- Station Währinger Straße und am Südtiroler Platz. Ein fixer Verkaufsplatz hat den Vorteil, dadurch Stammkund:innen gewinnen zu können, wenn es aber einer ist, der im Winter kalt ist, werde ich oft krank und kann nicht arbeiten. Ich möchte nicht betteln, mit dem Verkauf vom Augustin habe ich die Möglichkeit, einer Tätigkeit nachzugehen, anstatt um Almosen bitten zu müssen.
Geboren und aufgewachsen bin ich in Benin City, einer Drei-Millionen-Stadt im Süden Nigerias. Ich bin 41 Jahr

In [45]:
### POST REQUEST ###

url = "http://localhost:10004/wp-json/wp/v2/posts"

user = "lebe"
password = "gUwM J4pU sngD VHpk Cub7 quS2"

credentials = user + ":" + password

token = base64.b64encode(credentials.encode())

header = {"Authorization": "Basic " + token.decode("utf-8")}

post2 = {
        "title": title,
        "status": "publish",
        "content": readable_text,
        "excerpt": author + ", " + photograph + ", " + protocol,
        "post_type": "post",
        "featured_media": image_id,
    }

response = requests.post(url, headers=header, json=post2)

print("response", response.content)

response b'{"id":26,"date":"2023-06-29T14:37:02","date_gmt":"2023-06-29T14:37:02","guid":{"rendered":"http:\\/\\/localhost:10004\\/vom-gluck-eine-arbeit-zu-haben-2\\/","raw":"http:\\/\\/localhost:10004\\/vom-gluck-eine-arbeit-zu-haben-2\\/"},"modified":"2023-06-29T14:37:02","modified_gmt":"2023-06-29T14:37:02","password":"","slug":"vom-gluck-eine-arbeit-zu-haben-2","status":"publish","type":"post","link":"http:\\/\\/localhost:10004\\/vom-gluck-eine-arbeit-zu-haben-2\\/","title":{"raw":"Vom Gl\\u00fcck, eine Arbeit zu haben","rendered":"Vom Gl\\u00fcck, eine Arbeit zu haben"},"content":{"raw":"Vor zweieinhalb Jahren hatte ich eine schwierige Phase, da lief es nicht so gut f\\u00fcr mich, ich war arbeitslos und wegen der Corona- Pandemie konnte ich lange keine Arbeit finden. Bis mir Bekannte von der M\\u00f6glichkeit erz\\u00e4hlten, den Augustin zu verkaufen und so ein bisschen Geld zu verdienen. Dann habe ich eine Einschulung besucht und einen Augustin-Ausweis bekommen, seitdem verkauf